In [1]:
from hyperseti.data import from_fil, from_h5, DataArray

filpath = '/home/dancpr/blimpy/tests/test_data/Voyager1.single_coarse.fine_res.fil'
h5path = '/home/dancpr/blimpy/tests/test_data/Voyager1.single_coarse.fine_res.h5'
d_fil = from_fil(filpath)
d_h5  = from_h5(h5path)

bshuf filter already loaded, skip it.


In [2]:
str(d_h5)

"<DataArray: shape=(16, 1, 1048576), dims=('time', 'feed_id', 'frequency')>"

In [4]:
d_arr = d_h5.isel({'frequency': slice(0, 4096), 'time': slice(0, 10)})
d_arr.slice_info

(slice(0, 10, None), slice(None, None, None), slice(0, 4096, None))

In [8]:
ds = next(d_h5.iterate_through_data({'frequency': 2**16}))
str(ds)

"<DataArray: shape=(16, 1, 65536), dims=('time', 'feed_id', 'frequency')>"

In [9]:
for d_arr in d_h5.iterate_through_data({'frequency': 2**16}):
    print(d_arr.scales['frequency'])

<DimensionScale 'frequency': start 8421.386717353016 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8421.203611884266 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8421.020506415516 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8420.837400946766 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8420.654295478016 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8420.471190009266 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8420.288084540516 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8420.104979071766 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8419.921873603016 MHz step -2.7939677238464355e-06 MHz nstep 65536 >
<DimensionScale 'frequency': start 8419.738768134266 MHz step -2

In [4]:
import logbook
import hyperseti
hyperseti.logger_group.level = logbook.INFO
hyperseti.data.logger.level  = logbook.DEBUG

In [14]:
import itertools

dd = [slice(0, 524288, None)]
list(itertools.product(*dd))

TypeError: 'slice' object is not iterable

In [2]:
from hyperseti.hyperseti import find_et_serial

find_et_serial(h5path, gulp_size=2**20, 
               max_dd=1.0, min_dd=None, threshold=10, apply_normalization=True,
               n_boxcar=4, merge_boxcar_trials=True, min_ddistance=10, min_fdistance=50)

<DataArray: shape=(16, 1, 1048576), dims=('time', 'feed_id', 'frequency')>
## TOTAL TIME: 3.57s ##




,driftrate,f_start,snr,driftrate_idx,channel_idx,boxcar_size
0,0.009566,8419.921874,4093.418457,105.0,524288.0,1.0
27,0.401793,8419.921862,362.314087,146.0,524292.0,8.0
28,-0.564423,8419.921882,272.134552,45.0,524285.0,8.0
1,-0.459192,8419.921882,256.930725,56.0,524285.0,1.0
2,-0.736620,8419.921885,256.929291,27.0,524284.0,1.0
3,-0.248729,8419.921876,256.928314,78.0,524287.0,1.0
4,0.688787,8419.921862,256.927277,176.0,524292.0,1.0
7,-0.956649,8419.921890,256.907074,4.0,524282.0,1.0
8,0.975782,8419.921857,256.899231,206.0,524294.0,1.0
11,-0.822718,8419.921885,181.730331,18.0,524284.0,2.0


In [5]:
from hyperseti import run_pipeline

def find_et_serial(filename, filename_out='hits.csv', gulp_size=2**19, *args, **kwargs):
    ds = from_h5(filename)
    out = []
    for d_arr in ds.iterate_through_data({'frequency': gulp_size}):
        print(d_arr)
        d = d_arr.data
        f = d_arr.frequency
        t = d_arr.time
        md = {'fch1': f.val_start * f.units, 'df': f.val_step * f.units, 'dt': t.val_step * t.units}
        dedopp, metadata, hits = run_pipeline(d, md, *args, **kwargs)
        out.append(hits)
    
    dframe = pd.concat(out)
    dframe.to_csv(filename_out)
    
find_et_serial(h5path, max_dd=1.0, min_dd=None, threshold=100, apply_normalization=True,
                n_boxcar=4, merge_boxcar_trials=True, min_ddistance=10, min_fdistance=50)

<DataArray: shape=(16, 1, 524288), dims=('time', 'feed_id', 'frequency')>
[2021-04-15 14:10:48.716876] INFO: hyperseti.hyperseti: Normalisation time: 0.74ms
[2021-04-15 14:10:48.723980] INFO: hyperseti.hyperseti: --- Boxcar size: 1 ---
[2021-04-15 14:10:48.728367] INFO: hyperseti.hyperseti: Dedopp setup time: 0.87ms
[2021-04-15 14:10:48.732227] DEBUG: hyperseti.hyperseti: Kernel shape (grid, block)
[2021-04-15 14:10:48.736060] INFO: hyperseti.hyperseti: Dedopp kernel time: 3.88ms
[2021-04-15 14:10:48.738713] DEBUG: hyperseti.peak: Maxfilter: 1.91 ms
[2021-04-15 14:10:48.739603] DEBUG: hyperseti.peak: bitbash: 0.29 ms
[2021-04-15 14:10:49.114154] DEBUG: hyperseti.peak: coord search: 374.09 ms
[2021-04-15 14:10:49.115377] DEBUG: hyperseti.peak: crazyloop: 0.79 ms
[2021-04-15 14:10:49.115867] DEBUG: hyperseti.peak: prominent_peaks total: 1.29 ms
[2021-04-15 14:10:49.116368] INFO: hyperseti.hyperseti: Peak find time: 379.58ms
[2021-04-15 14:10:49.116876] INFO: hyperseti.hyperseti: Peak fin

In [6]:
md

NameError: name 'md' is not defined

In [90]:
d_sel

In [69]:
d_h5.data[isel(d_h5, s)]

0 time
1 feed_id
2 frequency


array([[4.7153664e+09, 5.0594243e+09, 5.2598994e+09, ..., 7.0528737e+09,
        4.8982861e+09, 5.1767214e+09],
       [4.0829606e+09, 5.7968077e+09, 6.0981827e+09, ..., 5.3419612e+09,
        4.4250788e+09, 5.7763446e+09],
       [6.0969042e+09, 4.5643085e+09, 4.8825871e+09, ..., 6.4770673e+09,
        5.0938399e+09, 5.0533325e+09],
       ...,
       [6.8482652e+09, 6.1711964e+09, 5.6285025e+09, ..., 5.5310628e+09,
        4.9069640e+09, 4.9941002e+09],
       [4.6555986e+09, 5.2430843e+09, 6.4172022e+09, ..., 5.0930427e+09,
        6.4045742e+09, 5.0717896e+09],
       [5.9711662e+09, 5.4580777e+09, 5.8471752e+09, ..., 5.6470390e+09,
        5.6385306e+09, 6.9158154e+09]], dtype=float32)